# Банки. Исследование надёжности заёмщиков

<h1>Содержание:<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Шаг-1.-Открываем-файл-с-данными-и-изучаем-общую-информацию" data-toc-modified-id="Шаг-1.-Открываем-файл-с-данными-и-изучаем-общую-информацию-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Шаг 1. Открываем файл с данными и изучаем общую информацию</a></span></li><li><span><a href="#Шаг-2.-Предобработка-данных" data-toc-modified-id="Шаг-2.-Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Шаг 2. Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Шаг-3.-Отвечаем-на-вопросы" data-toc-modified-id="Шаг-3.-Отвечаем-на-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Шаг 3. Отвечаем на вопросы</a></span></li><li><span><a href="#Шаг-4.-Общий-вывод" data-toc-modified-id="Шаг-4.-Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Шаг 4. Общий вывод</a></span></li></ul></div>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открываем файл с данными и изучаем общую информацию

In [70]:
#импортируем данные
import pandas as pd
reliability = pd.read_csv('/datasets/data.csv')

#смотрим названия колонок – приведены ли они к единому регистру, 
#нет ли лишних знаков, информативно ли их название. Смотрим количество значений в столбцах 
reliability.info()
#выводим первые 5 строк таблицы
reliability.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [71]:
#Определяем уникальные значения и их количество в столбце 'children'
reliability['children'].value_counts()
#Удаляем отрицательные значения и проверяем
reliability['children'] = reliability.children.abs()
reliability['children'].value_counts()

#Заменяем отрицательные значения на положительные в столбце со стажем
reliability['days_employed'] = reliability.days_employed.abs()
reliability.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Названия столбцов корректно названы и информативны:

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

На входе много мусора:
- В разных столбцах разное количество элементов --> в таблице есть пропущенные значения
- отрицательные значения в столбце 'days_employed'
- разные регистры в 'education'
- значения в столбцах 'days_employed' и 'total_income' неудобны для восприятия, усложняет восприятие таблицы
- в столбце 'children' есть отрицательное значение (исправляем, хотя есть вероятность, что это ярые представители чайлдфри :) ) и значение, выходящей за границы среднего - 20 детей (его исправлять не будем, т.к. возможно это семьи с приёмными детьми, хотя это может быть и ошибкой, но низкая доля этих значений не влияет на результат в целом). 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [72]:
#Проверяем долю отсутствующих значений
reliability.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

In [74]:
#На этом этапе заполняем нулями, в следующем шаге будет подробнее разбирать этот столбец
reliability['days_employed'] = reliability['days_employed'].fillna(0)

#определяем количество уникальных значений в столбце с типом занятости
types_activity = reliability['income_type'].unique()
types_activity

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [75]:
#создаём цикл для заполнения медианными значениями по группам
for i in types_activity:
    #присуждаем значение типу занятости
    activity = reliability.loc[reliability['income_type'] == i]
    #рассчитываем медиану в соответствии с типом занятости
    median_type = activity['total_income'].median()
    #заполняем строки с пропущенными значениями
    reliability.loc[(reliability['income_type'] == i) & (reliability['total_income'].isnull()), 'total_income'] = median_type

#Проверяем после долю отсутствующих значений
reliability.isna().mean()

children            0.0
days_employed       0.0
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
income_type         0.0
debt                0.0
total_income        0.0
purpose             0.0
dtype: float64

**Вывод**

Не хватает значений в столбцах 'days_employed' и 'total_income', ~10% от всей информации. 
Такие пропуски могли появиться из-за того, что клиенты не предоставили всю информацию о себе, но скорее всего это случилось на этапе автоматизированного сбора информации, т.к. в этих столбцах отсутствует одинаковое количество значений (2174), а информация из столбца 'total_income' важна для принятия решения о выдаче кредита

### Замена типа данных

In [59]:
#Переводим дни трудового стажа в года и меняем  типы данных в столбцах с float64 на int64
reliability['days_employed'] = (reliability['days_employed'] / 365).astype('int')
reliability['total_income'] = reliability['total_income'].astype('int')
#Переименовываем столбец со стажем
reliability.rename(columns={'days_employed': 'years_employed'}, inplace=True)

#Приводим к нижнему регистру строки в колонке 'education'
reliability['education'] = reliability['education'].str.lower()

#Определяем уникальные значения и их количество в столбце стажа
reliability['years_employed'].value_counts()
#Удаляем этот столбец
reliability.drop('years_employed', axis = 1, inplace=True)

#проверяем изменения, напечатав первые строки таблицы
reliability.head(5)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

В данных по стажу закралось много ошибок (отсутствующие и отрицательные значения), в том числе нереальные цифры стажа в годах - до 4000! В рамках данного исследования нам эти данные не нужны, но если нужно будет порверить гипотезу о зависимости трудового стажа и погашения кредита в срок, эти данные нужно будет заново запросить и понять на каком этапе происходят эти ошибки. Удаляем этот столбец


### Обработка дубликатов

In [78]:
#Определяем количество дубликатов-повторов
reliability.duplicated().sum()
#удаляем дубликаты, перезаписав индексы
reliability.drop_duplicates().reset_index(drop=True).sample(5) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19805,0,371428.655547,55,среднее,1,женат / замужем,0,F,пенсионер,0,123802.672334,ремонт жилью
11361,0,3150.935322,52,среднее,1,женат / замужем,0,F,госслужащий,0,170710.720136,операции с недвижимостью
14565,0,2731.992899,50,среднее,1,Не женат / не замужем,4,M,компаньон,0,262045.820909,покупка недвижимости
10465,3,0.000000,44,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,профильное образование
17520,0,236.991722,27,высшее,0,Не женат / не замужем,4,F,сотрудник,0,245311.912766,покупка жилой недвижимости


**Вывод**

Дубликаты удалены Возможно они появились по причинам:
•	ошибка выгрузки данных
•	один и тот же человек просил кредит
•	из-за "местного" глюка (например интернета) данные могли быть введены повторно

### Лемматизация

In [61]:
reliability['purpose']  = reliability['purpose'].apply(lambda purpose: ''.join(m.lemmatize(purpose)))
reliability.head(5)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилье\n
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиль\n
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилье\n
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительный образование\n
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьба\n


In [62]:
# Создаём функцию для группировки по целям кредита
def purpose_group (purpose):
    group = reliability['purpose']
    for row in group:
        if 'жиль' in purpose or 'недвиж' in purpose: 
            return 'недвижимость'
        elif 'авто'  in purpose:
            return 'автомобиль'
        elif 'образ'  in purpose:
            return 'образование'
        elif 'свадь'  in purpose:
            return 'свадьба'
        else:
            return 'неизвестно'


#Создаём отдельный столбец с целями кредита после группировки
reliability['purpose_new'] = reliability['purpose'].apply(purpose_group) 

#Проверяем, все ли значения у нас обработаны
reliability['purpose_new'].unique()#значения 'неизвестно' нет, значит функция purpose_group написана верно

#удалим столбец 'purpose'
reliability.drop('purpose', axis = 1, inplace=True)

#Определяем количество запросов кредита по каждой цели
reliability['purpose_new'].value_counts()

#Преверяем правильность работы кода со столбцами по целям кредита
reliability.head(5)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_new
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


**Вывод**

Проведя лемматизацию, выявили основные 4 цели, на которых запрашивали кредит и количество запросов по ним: 
недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348

### Категоризация данных

In [63]:
#Создаём новый столбец, в котором наличие детей будет обозначаться значением '1', а их отсутствие '0'
def have_child(children):
    if children == 0:
        return '0'
    return '1'
reliability['have_child'] = reliability['children'].apply(have_child)

#напишем функцию для группировки зарплаты
def group_salary(total_income):
    if total_income < 50000:
        return 'до 50'
    elif total_income > 50000 and total_income < 100000:
        return 'от 50 до 100'
    elif total_income > 100000 and total_income < 150000:
        return 'от 100 до 150'
    elif total_income > 150000 and total_income < 200000:
        return 'от 150 до 200'
    return 'более 250'
reliability['group_salary'] = reliability['total_income'].apply(group_salary)

reliability.head(5)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_new,have_child,group_salary
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,1,более 250
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,1,от 100 до 150
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,0,от 100 до 150
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,1,более 250
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,0,от 150 до 200


**Вывод**

Для удобства обработки информации, мы категоризировали данные по наличию детей и уровню дохода.

## Шаг 3. Отвечаем на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [64]:
#Группируем по тем, у кого есть задолженность
reliability.groupby('have_child')[['debt']].mean()

#Проверим гипотезу среди тех, у кого есть задолженность и дети, вдияет ли их количество на возврат
reliability.pivot_table(index='children',values='debt',aggfunc=['count','sum','mean']).reset_index()

,children,count,sum,mean
,,debt,debt,debt
0,0,14149,1063,0.075129
1,1,4865,445,0.091470
2,2,2055,194,0.094404
3,3,330,27,0.081818
4,4,41,4,0.097561
5,5,9,0,0.000000
6,20,76,8,0.105263


**Вывод**

Возврат кредита в срок - это отсутствие задолженностей по возврату кредита
Среди тех, у кого есть задолженность по кредиту: 9.2% имеют детей, 7,5% бездетные --> есть зависимость между наличием детей и возвратом кредита в срок
Гипотеза о зависимости между возвратом кредита и количестве детей не подтверждена

1. Для наглядности проверки гипотезы включила значение count(). Мы видим, что есть смысл сравнивать тут только тех, у кого либо один, либо два ребёнка, потому что по остальным группам значения слишком маленькие. Разница в наличии задолженности между теми, у кого один ребёнок и у кого два = 0.3% 
2. Делаем статистический тест https://www.evanmiller.org/ab-testing/chi-squared.html#

--> гипотеза о взаимосвязи количества детей и наличия задолженности не подтверждена 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [65]:
debt_family = reliability.groupby('family_status')[['debt']].mean()
debt_family.sort_values(by = 'debt', ascending = False)

,debt
family_status,
Не женат / не замужем,0.097405
гражданский брак,0.092890
женат / замужем,0.075202
в разводе,0.071130
вдовец / вдова,0.065625


Наименьшее количество задолженностей у вдовцов, наибольшее - у тех, кто не состоит в официальных отношениях

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [66]:
debt_salary = reliability.groupby('group_salary')[['debt']].mean()
debt_salary.sort_values(by = 'debt', ascending = False)

,debt
group_salary,
от 100 до 150,0.086403
от 150 до 200,0.084852
от 50 до 100,0.080909
более 250,0.070653
до 50,0.061828


Наименьшее количество должников среди тех, у кого самая низкая и самая высокая зарплата
В остальных группах (зарплата от 50 до 250) уровень дохода не влияет на возврат кредита в срок

- Как разные цели кредита влияют на его возврат в срок?

In [67]:
debt_purpose = reliability.groupby('purpose_new')[['debt']].mean()
debt_purpose.sort_values(by = 'debt', ascending = False)

,debt
purpose_new,
автомобиль,0.093395
образование,0.091994
свадьба,0.079216
недвижимость,0.072140


Больше всего должников среди тех, кто берёт кредит на автомобиль и образование, меньше всех - на недвижимость. 
--> риск выдачи кредитов на недвижимость ниже

Дополнительно проверим закономерности:
- Как пол заёмщика влияют на возврат кредита в срок?
- Как образование заёмщика влияют на возврат кредита в срок?

In [68]:
debt_gender = reliability.groupby('gender')[['debt']].mean()
debt_gender.sort_values(by = 'debt', ascending = False)

,debt
gender,
M,0.102497
F,0.069823
XNA,0.000000


In [69]:
debt_education = reliability.groupby('education')[['debt']].mean()
debt_education.sort_values(by = 'debt', ascending = False)

,debt
education,
начальное,0.109929
неоконченное высшее,0.091398
среднее,0.089542
высшее,0.052852
ученая степень,0.000000


**Вывод**

Процент должников среди мужчин 10, среди женщин - 7 --> женщины более надёжные заёмщики.
Чем выше уровень образования у заёмщика, тем ниже % невозврата кредита

## Шаг 4. Общий вывод

В работе нам нужно было определить "влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок".
Проведя анализ, мы сделали вывод:
- наличие детей влияет на факт погашения кредита в срок: среди должников 9.2% имеют детей, а 7,5% бездетные, а вот их количество  не играет принципиальной роли 
- семейное положение влияет на факт погашения кредита в срок: наименьшее количество задолженностей среди вдовцов, наибольшее - у тех, кто не состоит в официальных отношениях

Так же в процессе анализа были выявлены другие закономерности:
1. наличие задолженности зависит от уровня дохода:
    - ниже % должников среди тех, у кого уровень дохода менее 50 тыс. (6%) и более 250 тыс. (7%)
    - доля должников среди остальных зарплатных групп не разнится и в общей сложности составляет 25% от всех заёмщиков
2. Среди заёмщиков по 9% задолженности у тех заёмщиков, кто брал кредит на автомобиль и образование. у тех, кто брал кредит на недвижимость, % должников ниже - 7%
3. Мужчин, не выплативших вовремя кредит больше на 3%, чем женщин
4. Чем выше уровень образования, тем ниже % должников

Исходя из этих исследований, можно сделать вывод, что наименьший % должников среди женщин в разводе/вдов, без детей